In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Load the data
anime_data = pd.read_csv('anime.csv')
rating_data = pd.read_csv('rating.csv')

In [3]:
# anime_data = anime_data[anime_data['type'] == 'TV']
# anime_data = anime_data[anime_data['episodes'] != 'Unknown']
# anime_data = anime_data[anime_data['rating'] > 0]
# anime_data = anime_data.nlargest(700, 'members')
anime_data = anime_data.drop([2,4])
anime_data
# anime_id -- myanimelist.net's unique id identifying an anime.
# name -- full name of anime.
# genre -- comma separated list of genres for this anime.
# type -- movie, TV, OVA, etc.
# episodes -- how many episodes in this show. (1 if movie).
# rating -- average rating out of 10 for this anime.
# members -- number of community members that are in this anime's group

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [4]:
user_ids_to_delete = [28977, 9969]
condition = rating_data['user_id'].isin(user_ids_to_delete)
rating_data = rating_data[~condition]
rating_data
# user_id -- non identifiable randomly generated user id.
# anime_id -- the anime that this user has rated.
# rating -- rating out of 10 this user has assigned (-1 if the user watched it but didn't assign a rating).

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


In [5]:
rating_data = rating_data[rating_data.rating != -1]
Mean = rating_data.groupby(by="user_id",as_index=False)['rating'].mean()
Rating_avg = pd.merge(rating_data,Mean,on='user_id')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg = Rating_avg.rename(columns={'rating_x':'user_rating'})
Rating_avg = Rating_avg[['user_id','anime_id','user_rating', 'adg_rating']]
Rating_avg

,user_id,anime_id,user_rating,adg_rating
0,1,8074,10,0.000000
1,1,11617,10,0.000000
2,1,11757,10,0.000000
3,1,15451,10,0.000000
4,2,11771,10,0.000000
...,...,...,...,...
6337168,73515,16512,7,-1.547486
6337169,73515,17187,9,0.452514
6337170,73515,22145,10,1.452514
6337171,73516,790,9,0.000000


In [6]:
merged_datasets = pd.merge(anime_data, Rating_avg, on='anime_id', how='inner')
merged_datasets = merged_datasets[['user_id', 'anime_id', 'adg_rating']]
merged_datasets = merged_datasets.dropna(subset=['user_id'])
user_counts = merged_datasets['user_id'].value_counts()
top_500_users = user_counts.nlargest(500).index
user_item_matrix = merged_datasets[merged_datasets['user_id'].isin(top_500_users)].pivot_table(index='user_id', columns='anime_id', values='adg_rating')
#user_item_matrix = merged_datasets.pivot_table(index='user_id', columns='anime_id', values='rating')
user_item_matrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,34119,34136,34238,34240,34283,34324,34325,34349,34367,34475
user_id,,,,,,,,,,,,,,,,,,,,,
226,0.319407,NaN,0.319407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
446,1.763122,0.763122,1.763122,NaN,NaN,NaN,NaN,NaN,NaN,2.763122,...,NaN,NaN,NaN,1.763122,NaN,NaN,NaN,NaN,NaN,NaN
478,NaN,NaN,NaN,NaN,NaN,NaN,-0.513326,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
958,2.565773,2.565773,NaN,NaN,NaN,NaN,NaN,NaN,1.565773,2.565773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1145,NaN,NaN,0.243363,NaN,NaN,NaN,0.243363,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70953,-0.404819,NaN,-0.404819,NaN,NaN,1.595181,NaN,NaN,-0.404819,1.595181,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.658777,NaN,NaN,NaN
73135,2.289047,2.289047,NaN,NaN,NaN,NaN,4.289047,NaN,NaN,3.289047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
user_item_matrix = user_item_matrix.fillna(user_item_matrix.mean(axis=0))
#user_item_matrix.apply(lambda row: row.fillna(row.mean()), axis=0)
user_item_matrix

anime_id,1,5,6,7,8,15,16,17,18,19,...,34119,34136,34238,34240,34283,34324,34325,34349,34367,34475
user_id,,,,,,,,,,,,,,,,,,,,,
226,0.319407,0.974595,0.319407,0.032048,-0.261487,0.777386,1.009829,0.193102,0.861345,1.733119,...,-2.264725,0.912302,0.430125,0.785972,-0.469387,-1.965521,0.350297,-0.043927,-1.630457,-0.998688
446,1.763122,0.763122,1.763122,0.032048,-0.261487,0.777386,1.009829,0.193102,0.861345,2.763122,...,-2.264725,0.912302,0.430125,1.763122,-0.469387,-1.965521,0.350297,-0.043927,-1.630457,-0.998688
478,1.367889,0.974595,0.950713,0.032048,-0.261487,0.777386,-0.513326,0.193102,0.861345,1.733119,...,-2.264725,0.912302,0.430125,0.785972,-0.469387,-1.965521,0.350297,-0.043927,-1.630457,-0.998688
958,2.565773,2.565773,0.950713,0.032048,-0.261487,0.777386,1.009829,0.193102,1.565773,2.565773,...,-2.264725,0.912302,0.430125,0.785972,-0.469387,-1.965521,0.350297,-0.043927,-1.630457,-0.998688
1145,1.367889,0.974595,0.243363,0.032048,-0.261487,0.777386,0.243363,0.193102,0.861345,1.733119,...,-2.264725,0.912302,0.430125,0.785972,-0.469387,-1.965521,0.350297,-0.043927,-1.630457,-0.998688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70953,-0.404819,0.974595,-0.404819,0.032048,-0.261487,1.595181,1.009829,0.193102,-0.404819,1.595181,...,-2.264725,0.912302,0.430125,0.785972,-0.469387,-1.965521,0.350297,-0.043927,-1.630457,-0.998688
71792,1.367889,0.974595,0.950713,0.032048,-0.261487,0.777386,1.009829,0.193102,0.861345,1.733119,...,-2.264725,0.912302,0.430125,0.785972,-0.469387,-1.965521,0.658777,-0.043927,-1.630457,-0.998688
73135,2.289047,2.289047,0.950713,0.032048,-0.261487,0.777386,4.289047,0.193102,0.861345,3.289047,...,-2.264725,0.912302,0.430125,0.785972,-0.469387,-1.965521,0.350297,-0.043927,-1.630457,-0.998688


In [8]:
def calculate_similarity_matrix(matrix):
    num_users, num_items = matrix.shape
    similarity_matrix = np.zeros((num_users, num_users))

    for i in range(num_users):
        for j in range(i, num_users):
            vec1 = matrix[i, :]
            vec2 = matrix[j, :]
            similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity

    return similarity_matrix


In [9]:
# 1. Calculate similarity_matrix using cosine similarity based on user-item matrix. DONE
# 2. Select target_user. DONE
# 3. Retrieve ratings given by target_user from user-item matrix. DONE
# 4. Find most similar_users to target_user based on similarity_matrix using the neighborhood size. DONE
# 5. Retrieve ratings given by similar_users to the items.
# 6. Predict ratings for target_user using weighted average of similar_users' ratings and their similarity values.
# 7. Select top_items with highest predicted ratings (e.g., top 10).
# 8. Retrieve anime_names for the selected items from the anime dataset.
# 9. Output the top_items and anime_names as recommendations.


In [10]:
similarity_matrix = calculate_similarity_matrix(user_item_matrix.values)
similarity_matrix = pd.DataFrame(similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)
similarity_matrix

user_id,226,446,478,958,1145,1344,1456,1497,1530,2632,...,69964,70015,70161,70507,70725,70953,71792,73135,73234,73378
user_id,,,,,,,,,,,,,,,,,,,,,
226,1.000000,0.893591,0.919905,0.906863,0.911861,0.916710,0.912488,0.875455,0.849753,0.921230,...,0.919550,0.928533,0.884475,0.942586,0.875803,0.926512,0.897228,0.798682,0.919643,0.927910
446,0.893591,1.000000,0.914756,0.904756,0.899604,0.900883,0.905540,0.868012,0.830830,0.913560,...,0.908930,0.921752,0.869455,0.935898,0.874059,0.914814,0.891336,0.793278,0.913383,0.926955
478,0.919905,0.914756,1.000000,0.932398,0.932253,0.933838,0.939993,0.899635,0.860785,0.939895,...,0.938388,0.952295,0.908704,0.963160,0.901152,0.943641,0.920131,0.818604,0.942148,0.954016
958,0.906863,0.904756,0.932398,1.000000,0.915107,0.919989,0.925720,0.890596,0.840873,0.921854,...,0.923178,0.939657,0.886819,0.950277,0.884372,0.933264,0.905110,0.806298,0.931641,0.940808
1145,0.911861,0.899604,0.932253,0.915107,1.000000,0.920220,0.922514,0.882099,0.842075,0.921101,...,0.925279,0.936468,0.892261,0.947610,0.899955,0.933499,0.910525,0.813576,0.924778,0.945455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70953,0.926512,0.914814,0.943641,0.933264,0.933499,0.938428,0.942247,0.897804,0.861853,0.936495,...,0.937766,0.952176,0.900705,0.962542,0.897443,1.000000,0.920560,0.822301,0.937026,0.957427
71792,0.897228,0.891336,0.920131,0.905110,0.910525,0.913221,0.917981,0.877995,0.838942,0.914973,...,0.913587,0.928740,0.882486,0.937660,0.888146,0.920560,1.000000,0.807168,0.918369,0.933123
73135,0.798682,0.793278,0.818604,0.806298,0.813576,0.810381,0.810945,0.777304,0.736620,0.813465,...,0.810335,0.828464,0.784160,0.837875,0.788174,0.822301,0.807168,1.000000,0.815853,0.830392


In [11]:
ratings = pd.read_csv('rating.csv')
def check_rating_exists(user_id, anime_id):
    # Load the rating data into a DataFrame
    # Check if a rating exists for the given user_id and anime_id
    rating_exists = ratings[(ratings['user_id'] == user_id) & (ratings['anime_id'] == anime_id)]['rating'].notna().any()
    return rating_exists

In [12]:
animes = pd.read_csv('anime.csv')  # Replace with the path to your anime dataset file

def generate_recommendations(neighborhood_size):
    user_input = input("Enter a user ID: ")
    target_user_id = int(user_input)
    # This line retrieves the similarity values between the target user and other users from the similarity matrix. It selects the most similar users to the target user based on the cosine similarity scores.
    similar_user_indices = similarity_matrix[target_user_id].sort_values(ascending=False)[1:neighborhood_size+1].index
    #This line retrieves the ratings given by the similar users to the items in the user-item matrix.
    similar_user_ratings = user_item_matrix.loc[similar_user_indices]
    #This line retrieves the similarity values between the target user and the similar users from the similarity matrix.
    user_similarities = similarity_matrix.loc[target_user_id, similar_user_indices]
    #This line calculates the predicted ratings for the target user using the weighted average of the similar users' ratings and their similarity values.
    predicted_ratings = np.dot(user_similarities, similar_user_ratings) / np.sum(np.abs(user_similarities))
    #This line creates a pandas Series object with the predicted ratings for the target user. The index of the Series object is the item IDs from the user-item matrix.
    predicted_ratings_series = pd.Series(predicted_ratings, index=user_item_matrix.columns)
    #The rest of the code is used to retrieve the top items with the highest predicted ratings and output the anime names and other features as recommendations.
    top_items_indices = predicted_ratings_series.sort_values(ascending=False)[:70].index
    top_10_animes = []
    for i in top_items_indices:
        if len(top_10_animes) <= 10:
            if check_rating_exists(target_user_id, i) == False:
                top_10_animes.append(i)
    top_anime_names = animes.loc[animes['anime_id'].isin(top_items_indices)]['name'].values
    print(f"Top Recommendations for User {target_user_id}: \n")
    filtered_anime_data = animes[animes['name'].isin(top_anime_names)]
    new_df = filtered_anime_data[['name', 'genre', 'type', 'episodes', 'rating']]
    return new_df.head(10)

In [13]:
def get_recommendations():
    return generate_recommendations(neighborhood_size=25)

In [14]:
#run this code cell to get recommendations for a user
#input: user_id (ex: 226,  446,  478,  958, 1145, 1344, 1456, 1497, 1530, 2632, 2951, 3040, 4468, 5073, 5137, 5264, 5310)
get_recommendations()

Top Recommendations for User 1145: 



,name,genre,type,episodes,rating
0,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37
1,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26
3,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17
5,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15
6,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13
7,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11
8,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10
9,Gintama: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11
11,Koe no Katachi,"Drama, School, Shounen",Movie,1,9.05
12,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,201,9.04


In [16]:
# def k_fold_cross_validation(matrix, k, neighborhood_sizes):
#     num_users, num_items = matrix.shape
#     fold_size = num_users // k
#     results = {}

#     for neighborhood_size in neighborhood_sizes:
#         avg_rmse = 0.0
#         print('neighborhood_size', neighborhood_size)
#         for fold in range(k):
#             # Split the data into training and testing sets
#             start = fold * fold_size
#             end = (fold + 1) * fold_size
#             test_data = matrix[start:end, :]

#             # Create the similarity matrix using the training data
#             training_data = np.concatenate((matrix[:start, :], matrix[end:, :]), axis=0)
#             similarity_matrix = calculate_similarity_matrix(training_data)
#             # Predict ratings for the test data
#             predicted_ratings = np.zeros(test_data.shape)
#             for i in range(test_data.shape[0]):
#                 user_vector = test_data[i, :]
#                 user_indices = np.argsort(similarity_matrix[i, :])[::-1][:neighborhood_size]
#                 user_similarities = similarity_matrix[i, user_indices]
#                 user_ratings = training_data[user_indices, :]
#                 predicted_ratings[i, :] = np.dot(user_similarities, user_ratings) / np.sum(np.abs(user_similarities))

#             # Calculate RMSE for the fold
#             rmse = np.sqrt(np.mean((test_data - predicted_ratings) ** 2))
#             avg_rmse += rmse

#         # Calculate average RMSE across all folds
#         avg_rmse /= k
#         results[neighborhood_size] = avg_rmse

#     return results

# k = 5
# neighborhood_sizes = [5, 10, 15, 20, 25]
# results = k_fold_cross_validation(user_item_matrix.values, k, neighborhood_sizes)

# # Print the results
# for neighborhood_size, rmse in results.items():
#     print(f"Neighborhood Size: {neighborhood_size}, RMSE: {rmse}")


#Output: Best Neighborhood Size: 25, RMSE: 1.001
